In [2]:
! pip install nba_api

import pandas as pd
from nba_api.stats.endpoints import commonplayerinfo, leaguedashplayerstats
from tqdm import tqdm
from time import sleep
from configs.project_config import SEASON, RAW_CONTEXT_FILE

def get_active_player_ids(season: str):
    """Fetch all active player IDs for the given season."""
    stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season)
    df = stats.get_data_frames()[0]
    return df[['PLAYER_ID', 'PLAYER_NAME']]

def get_player_context(player_id: int):
    """Fetch biographical/contextual data for a single player."""
    try:
        info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        df = info.get_data_frames()[0]
        record = {
            "PLAYER_ID": df.loc[0, "PERSON_ID"],
            "PLAYER_NAME": df.loc[0, "DISPLAY_FIRST_LAST"],
            "BIRTHDATE": df.loc[0, "BIRTHDATE"],
            "COUNTRY": df.loc[0, "COUNTRY"],
            "DRAFT_YEAR": df.loc[0, "DRAFT_YEAR"],
            "DRAFT_ROUND": df.loc[0, "DRAFT_ROUND"],
            "DRAFT_NUMBER": df.loc[0, "DRAFT_NUMBER"],
        }
        return record
    except Exception as e:
        print(f"Error fetching player {player_id}: {e}")
        return None

def main():
    print(f"Fetching NBA context data for season {SEASON}...")

    players = get_active_player_ids(SEASON)
    context_data = []

    for _, row in tqdm(players.iterrows(), total=len(players)):
        record = get_player_context(row.PLAYER_ID)
        if record:
            context_data.append(record)
        sleep(0.6)  # To avoid rate-limiting

    df_context = pd.DataFrame(context_data)
    df_context.to_csv(RAW_CONTEXT_FILE, index=False)
    print(f"Saved context data to {RAW_CONTEXT_FILE}")

if __name__ == "__main__":
    main()



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Fetching NBA context data for season 2024-25...


100%|██████████| 569/569 [06:56<00:00,  1.37it/s]

Saved context data to /Users/albertoramirez/Documents/GitHub/Analysis-of-NBA-Contract-Valuation-Bias/data/raw/raw_player_context.csv
